In [14]:
import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed
import tushare as ts
from data_tools.api import *
from utilscht.Data import *
import pymysql
import click

DB_INFO = dict(host='192.168.1.234',
               user='winduser',
               password='1qaz@WSX',
               db='wind')

conn = pymysql.connect(**DB_INFO, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

In [16]:
# current function to get 1min_freq data
def get_stk_data(sid, date):
    
    try:
        raw_bar_dtype = [
            ('sid', '<U16'),
            ('DataDate', 'i4'),
            ('ticktime', 'i4'),
            ('pre_close', 'f8'),
            ('open', 'f8'),
            ('high', 'f8'),
            ('low', 'f8'),
            ('close', 'f8'),
            ('volume', 'f8'),
            ('amount', 'f8'),
            ('bid_amount', 'f8'),
            ('ask_amount', 'f8'),
            ('vwap', 'f8'),
            ('twap', 'f8'),
            ('ret', 'f8')
        ]


        bar=np.memmap(r"/share/intern_share/dat/stk_bar/1min/2019/{}/{}.dat".format(date,sid),dtype=raw_bar_dtype,mode="r")
        stock_data=pd.DataFrame(bar)[["sid","DataDate","ticktime","open", "high", "low", "close","pre_close"]]
        stock_data["ticktime"]=stock_data.index
    except:
        try:
            stock_data = get_stk_bar(sid, freq="1m", start=date, end=date, fields=["open", "high", "low", "close"])
            stock_data = stock_data.reset_index().rename(columns={"index": "datetime"})
            stock_data["ticktime"] = stock_data["datetime"].index
            stock_data["DataDate"] = stock_data["datetime"].apply(lambda x:str(x.date()))
            del stock_data["datetime"]
            stock_data["pre_close"] = get_stk_bar(sid, freq="1d", start=date, end=date, fields=["pre_close"]).values[0][0]
            stock_data["sid"] = sid
        except:
            logging.warning("Data Lost for {} on {}".format(sid,date))
    stock_data["10min_change"]=stock_data["close"].pct_change(10)        
    return stock_data[stock_data["10min_change"]<-0.02]


def get_daily_data(date):
    df=query_table("DailyBar",start_date=date,end_date=date,fields=["tradable"])
    stk_ls=list(df["sid"][df["tradable"]==1])

    results = Parallel(n_jobs=16, verbose=5, backend="loky", batch_size='auto') \
        (delayed(get_stk_data)(sid, date) for sid in stk_ls)
    data = pd.concat(results)

    return data.sort_values(["sid","ticktime"])

In [20]:
date_ls=get_trade_dates(20190101,20191029)

summary_df=pd.DataFrame()
for date in date_ls:
    print(date,end="\r")
    daily_data=get_daily_data(date)
    summary_df=pd.concat([summary_df,daily_data])
    
summary_df

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    1.9s
[Parallel(n_jobs=16)]: Done 1168 tasks      | elapsed:    4.1s
[Parallel(n_jobs=16)]: Done 2464 tasks      | elapsed:    7.0s
[Parallel(n_jobs=16)]: Done 3553 out of 3553 | elapsed:    9.3s finished


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.1s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    1.6s
[Parallel(n_jobs=16)]: Done 1168 tasks      | elapsed:    3.7s
[Parallel(n_jobs=16)]: Done 2464 tasks      | elapsed:    6.4s
[Parallel(n_jobs=16)]: Done 3554 out of 3554 | elapsed:    8.7s finished


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.1s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    1.6s
[Parallel(n_jobs=16)]: Done 1168 tasks      | elapsed:    4.1s
[Parallel(n_jobs=16)]: Done 2464 tasks      | elapsed:    6.9s
[Parallel(n_jobs=16)]: Done 3554 out of 3554 | elapsed:    9.1s finished


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.1s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    1.8s
[Parallel(n_jobs=16)]: Done 720 tasks      | elapsed:    2.9s
[Parallel(n_jobs=16)]: Done 1368 tasks      | elapsed:    4.3s
[Parallel(n_jobs=16)]: Done 2160 tasks      | elapsed:    5.9s
[Parallel(n_jobs=16)]: Done 3096 tasks      | elapsed:    7.9s
[Parallel(n_jobs=16)]: Done 3553 out of 3553 | elapsed:    8.8s finished


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.1s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 720 tasks      | elapsed:    2.8s
[Parallel(n_jobs=16)]: Done 1368 tasks      | elapsed:    4.1s
[Parallel(n_jobs=16)]: Done 2160 tasks      | elapsed:    5.8s
[Parallel(n_jobs=16)]: Done 3096 tasks      | elapsed:    7.8s
[Parallel(n_jobs=16)]: Done 3525 out of 3556 | elapsed:    8.7s remaining:    0.1s
[Parallel(n_jobs=16)]: Done 3556 out of 3556 | elapsed:    8.7s finished


...

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.5s
[Parallel(n_jobs=16)]: Done 164 tasks      | elapsed:    2.2s
[Parallel(n_jobs=16)]: Done 448 tasks      | elapsed:    3.0s
[Parallel(n_jobs=16)]: Done 1680 tasks      | elapsed:    6.7s
[Parallel(n_jobs=16)]: Done 3264 tasks      | elapsed:   11.5s
[Parallel(n_jobs=16)]: Done 3599 out of 3599 | elapsed:   12.4s finished


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.5s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    2.1s
[Parallel(n_jobs=16)]: Done 256 tasks      | elapsed:    3.0s
[Parallel(n_jobs=16)]: Done 436 tasks      | elapsed:    4.1s
[Parallel(n_jobs=16)]: Done 848 tasks      | elapsed:    5.6s
[Parallel(n_jobs=16)]: Done 1784 tasks      | elapsed:    8.6s
[Parallel(n_jobs=16)]: Done 2864 tasks      | elapsed:   12.9s
[Parallel(n_jobs=16)]: Done 3564 out of 3595 | elapsed:   15.1s remaining:    0.1s
[Parallel(n_jobs=16)]: Done 3595 out of 3595 | elapsed:   15.2s finished


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.1s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 1168 tasks      | elapsed:    3.8s
[Parallel(n_jobs=16)]: Done 2464 tasks      | elapsed:    6.5s
[Parallel(n_jobs=16)]: Done 3544 tasks      | elapsed:    8.9s
[Parallel(n_jobs=16)]: Done 3596 out of 3596 | elapsed:    9.0s finished


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.1s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    1.7s


KeyboardInterrupt: 

In [24]:
sid_date_df=new_summary_df.drop_duplicates(subset=("sid","DataDate"),keep="first")[["sid","DataDate"]]
sid_date_df.sort_values(["DataDate","sid"],inplace=True)
sid_date_df.reset_index(drop=True,inplace=True)
sid_date_df.to_csv(r"data/10mindecre_>2%_stocklist.csv",index=False)

In [30]:
sid_date_df.to_csv(r"/share/intern_share/assign_1030/10mindecre_>2%_stocklist.csv",index=False)

In [31]:
sid_date_df

,sid,DataDate
0,000017.SZ,20190102
1,000058.SZ,20190102
2,000069.SZ,20190102
3,000403.SZ,20190102
4,000409.SZ,20190102
5,000410.SZ,20190102
6,000525.SZ,20190102
7,000547.SZ,20190102
8,000551.SZ,20190102
9,000586.SZ,20190102


In [ ]:
import numpy as np
import pandas as pd
import os
DATA_ROOT = "/share/xfzhang/store_data/"

__all__ = ['trade_calendar', 'stk_description', 'idx_description', 'df_sid_change']


def load_calandar_descr():
    path = os.path.join(DATA_ROOT, 'base_data/calendar')
    with open(path, 'r') as f:
        dates = [line.strip() for line in f]
    arr_calendar = np.array(dates).astype('i4')
            
    path = os.path.join(DATA_ROOT, 'base_data/stock_description')
    with open(path, 'r') as f:
        stk_descr = [line.strip().split(' ') for line in f]
    df_stk_dscr = pd.DataFrame(stk_descr, columns=['sid', 'list_date', 'delist_date'])
    df_stk_dscr['list_date'] = df_stk_dscr['list_date'].values.astype('i4')
    df_stk_dscr['delist_date'] = df_stk_dscr['delist_date'].values.astype('i4')
            
    path = os.path.join(DATA_ROOT, 'base_data/index_description')
    with open(path, 'r') as f:
        idx_descr = [line.strip().split(' ') for line in f]
    df_idx_dscr = pd.DataFrame(idx_descr, columns=['iid', 'list_date', 'expire_date'])
    df_idx_dscr['list_date'] = df_idx_dscr['list_date'].values.astype('i4')
    df_idx_dscr['expire_date'] = df_idx_dscr['expire_date'].values.astype('i4')
    
    path = os.path.join(DATA_ROOT, 'base_data/sid_change')
    with open(path, 'r') as f:
        sid_change = [line.strip().split(' ') for line in f]
    df_sid_change = pd.DataFrame(sid_change, columns=['date', 'old_sid', 'new_sid'])
    df_sid_change['date'] = df_sid_change['date'].values.astype('i4')
            
    return arr_calendar, df_stk_dscr, df_idx_dscr, df_sid_change

arr_calendar, df_stk_dscr, df_idx_dscr, df_sid_change = load_calandar_descr()


class TradeCalendar:
    def __init__(self):
        self.trade_dates = arr_calendar

    def get_prev_trade_date(self, date: int, n=1):
        pos = self.trade_dates.searchsorted(int(date))
        if pos < n:
            raise MemoryError
        return self.trade_dates[pos-n]

    def get_next_trade_date(self, date: int):
        pos = self.trade_dates.searchsorted(int(date), side='right')
        return self.trade_dates[pos]

    def get_trade_dates(self, start_date, end_date):
        s = np.searchsorted(self.trade_dates, int(start_date))
        e = np.searchsorted(self.trade_dates, int(end_date), side='right')
        return self.trade_dates[s:e].copy()
    

class StockDescription():
    def __init__(self):
        self.data = df_stk_dscr
        
    def get_dates_sids(self, start_date, end_date=None, n=0):  #list date>=n, not include start_date
        end_date = start_date if end_date is None else end_date
        start_date, end_date = int(start_date), int(end_date)
        s_pos = np.searchsorted(trade_calendar.trade_dates, start_date)
        e_pos = np.searchsorted(trade_calendar.trade_dates, end_date, side='right') - 1
        assert s_pos >= n and s_pos <= e_pos
        start_date0 = trade_calendar.trade_dates[s_pos - n]
        end_date0 = trade_calendar.trade_dates[e_pos - n]
        
        df_raw = self.data
        unique_dates = trade_calendar.get_trade_dates(start_date0, end_date)
        df = df_raw[(df_raw.list_date <= end_date0) & (df_raw.delist_date > start_date)]
        unique_sids = df.sid.values.astype('U9')
        
        arr = np.zeros([len(unique_dates), len(unique_sids)], dtype='i4')
        pos = np.searchsorted(unique_sids, df.sid)
        pos_up = np.searchsorted(unique_dates, df.list_date)
        pos_dn = np.searchsorted(unique_dates, df.delist_date)
        for i in range(len(df)):
            #if pos_up[i] + n < len(arr):
            arr[pos_up[i] + n, pos[i]] += 1
            if pos_dn[i] < len(arr):
                arr[pos_dn[i], pos[i]] -= 1
        
        arr_cum = np.cumsum(arr, axis=0)
        dates_pos, sids_pos = np.where(arr_cum[n:]>=1)
        # dates = unique_dates[dates_pos]
        # sids = unique_sids[sids_pos]
        return unique_dates[n:], unique_sids, dates_pos, sids_pos


class IndexDescription:
    def __init__(self):
        self.data = df_idx_dscr

    def get_dates_iids(self, start_date, end_date=None):
        end_date = start_date if end_date is None else end_date
        start_date, end_date = int(start_date), int(end_date)
        df_raw = self.data
        unique_dates = trade_calendar.get_trade_dates(start_date, end_date)
        df = df_raw[(df_raw.list_date <= end_date) & (df_raw.expire_date > start_date)]
        unique_iids = df.iid.values.astype('U9')
        
        arr = np.zeros([len(unique_dates), len(unique_iids)], dtype='i4')
        pos = np.searchsorted(unique_iids, df.iid)
        pos_up = np.searchsorted(unique_dates, df.list_date)
        pos_dn = np.searchsorted(unique_dates, df.expire_date)
        for i in range(len(df)):
            if pos_up[i] < len(arr):
                arr[pos_up[i], pos[i]] += 1
            if pos_dn[i] < len(arr):
                arr[pos_dn[i], pos[i]] -= 1
        
        arr_cum = np.cumsum(arr, axis=0)
        dates_pos, iids_pos = np.where(arr_cum>=1)
        # dates = unique_dates[dates_pos]
        # iids = unique_iids[iids_pos]
        return unique_dates, unique_iids, dates_pos, iids_pos  

    
trade_calendar = TradeCalendar()
stk_description = StockDescription()
idx_description = IndexDescription()

if __name__ == '__main__':
    _, sids = stk_description.get_sids('20181228')
    _, iids = idx_description.get_iids('20181228')
    print(len(sids), len(iids))

In [5]:
import numpy as np
import pandas as pd
import os
import bcolz


col_dtype_dflt = [('date', 'i4', 0), ('sid', 'S9', b'000000000'),  ('time', 'i4', 0),
                  
                  ('pre_close', 'f8', np.nan), ('open', 'f8', np.nan), ('high', 'f8', np.nan), 
                  ('low', 'f8', np.nan), ('close', 'f8', np.nan), ('amount', 'f8', np.nan), ('vwap', 'f8', np.nan),
                  
                  ('amount_mean', 'f8', 0.0), ('amount_std', 'f8', 0.0),  # ('trans_num', 'i8', 0), 
                  ('ask_id_amount_mean', 'f8', 0.0), ('ask_id_amount_std', 'f8', 0.0),  # ('ask_id_amount_mean', 'f8', 0.0), ('bid_id_trans_num', 'i8', 0), 
                  ('bid_id_amount_mean', 'f8', 0.0), ('bid_id_amount_std', 'f8', 0.0),
                  
                  ('act_ask_amount_sum', 'i8', 0),  ('act_ask_amount_mean', 'f8', 0.0), ('act_ask_amount_std', 'f8', 0.0),
                  ('act_bid_amount_sum', 'i8', 0),  ('act_bid_amount_mean', 'f8', 0.0), ('act_bid_amount_std', 'f8', 0.0),
                  ('act_ask_id_amount_mean', 'f8', 0.0), ('act_ask_id_amount_std', 'f8', 0.0),
                  ('act_bid_id_amount_mean', 'f8', 0.0), ('act_bid_id_amount_std', 'f8', 0.0),
                  
                  ('tot_ask_amount1', 'f8', np.nan), ('avg_ask_price1', 'f8', np.nan),
                  ('tot_bid_amount1', 'f8', np.nan), ('avg_bid_price1', 'f8', np.nan),
                  ('tot_ask_amount2', 'f8', np.nan), ('avg_ask_price2', 'f8', np.nan),
                  ('tot_bid_amount2', 'f8', np.nan), ('avg_bid_price2', 'f8', np.nan),
                  ('tot_ask_amount5', 'f8', np.nan), ('avg_ask_price5', 'f8', np.nan),
                  ('tot_bid_amount5', 'f8', np.nan), ('avg_bid_price5', 'f8', np.nan),
                  ('tot_ask_amount10', 'f8', np.nan), ('avg_ask_price10', 'f8', np.nan),
                  ('tot_bid_amount10', 'f8', np.nan), ('avg_bid_price10', 'f8', np.nan),
                  ('tot_ask_amount', 'f8', np.nan), ('avg_ask_price', 'f8', np.nan),
                  ('tot_bid_amount', 'f8', np.nan), ('avg_bid_price', 'f8', np.nan),
                  
                  ('close_suspend', 'i4', 0), ('close_limit', 'i4', 0), ('all_limit', 'i4', 0),
                 ]

col_dtype_dflt_idx = [('date', 'i4', 0), ('iid', 'S9', b'000000000'),  ('time', 'i4', 0),
                  
                  ('pre_close', 'f8', np.nan), ('open', 'f8', np.nan), ('high', 'f8', np.nan), 
                  ('low', 'f8', np.nan), ('close', 'f8', np.nan), ('volume', 'f8', np.nan), ('amount', 'f8', np.nan),
                 ]


def get_db(start_date, end_date, cols=None): 
    path = f'{DATA_ROOT}/bcolz_data/1d/stock_data'
    table = bcolz.open(path, mode='r')
    dates = table['date']
    columns = [ele for ele in table.names if ele != '_id']  # bcolz
    s = np.searchsorted(dates, int(start_date))
    e = np.searchsorted(dates, int(end_date), side='right')
    if cols is not None:
        assert len(set(cols).difference(columns)) == 0
        cols = [ele for ele in columns if ele in ['sid', 'date'] + cols]
    else:
        cols = columns
    df = pd.DataFrame(table[cols][s:e])
    df.sid = df.sid.values.astype('U9')
    return df


def get_idx_db(start_date, end_date, cols=None): 
    path = f'{DATA_ROOT}/bcolz_data/1d/index_data'
    table = bcolz.open(path, mode='r')
    dates = table['date']
    columns = [ele for ele in table.names if ele != '_id']  # bcolz
    s = np.searchsorted(dates, int(start_date))
    e = np.searchsorted(dates, int(end_date), side='right')
    if cols is not None:
        assert len(set(cols).difference(columns)) == 0
        cols = [ele for ele in columns if ele in ['iid', 'date'] + cols]
    else:
        cols = columns
    df = pd.DataFrame(table[cols][s:e])
    df.iid = df.iid.values.astype('U9')
    return df


def get_mb(start_date, end_date,  freq='15m', fields=None): 
    trade_dates = trade_calendar.get_trade_dates(start_date, end_date)
    path = f"{DATA_ROOT}/np_data/{freq}/stock_data"
    dtype = [(ele[0], ele[1]) for ele in col_dtype_dflt]
    arr_all = []
    if fields is None:
        fields = [ele[0] for ele in dtype]
    fields2 = [ele[0] for ele in dtype if ele[0] in fields+['date', 'sid', 'time']]
    for i, date in enumerate(trade_dates):
        year = date // 10000
        path2 = f"{path}/{year}/{date}.dat"
        arr = np.memmap(path2, dtype=dtype, mode='r')
        arr_all.append(arr[fields2])
    arr_all = np.concatenate(arr_all)
    return arr_all

def get_idx_mb(start_date, end_date,  freq='15m', fields=None): 
    trade_dates = trade_calendar.get_trade_dates(start_date, end_date)
    path = f"{DATA_ROOT}/np_data/{freq}/index_data"
    dtype = [(ele[0], ele[1]) for ele in col_dtype_dflt_idx]
    arr_all = []
    if fields is None:
        fields = [ele[0] for ele in dtype]
    fields2 = [ele[0] for ele in dtype if ele[0] in fields+['date', 'iid', 'time']]
    for i, date in enumerate(trade_dates):
        year = date // 10000
        path2 = f"{path}/{year}/{date}.dat"
        arr = np.memmap(path2, dtype=dtype, mode='r')
        arr_all.append(arr[fields2])
    arr_all = np.concatenate(arr_all)
    return arr_all